# Collaborative Filtering Recommender Systems

## Imports

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Reading Movie Data

In [18]:
movies_df = pd.read_csv("movies.csv")
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Preprocessing Movie Data

In [ ]:
movies_df["year"] = movies_df.title.str.extract(r"(\(\d{4}\))", expand=False)
movies_df["year"] = movies_df.year.str.extract(r"(\d{4})", expand=False)

movies_df["title"] = movies_df.title.str.replace(r" (\(\d{4}\))", "", regex=True)

movies_df = movies_df.drop("genres", axis=1)
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


## Reading Rating Data

In [20]:
ratings_df = pd.read_csv("ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


## Preprocessing Rating Data

In [21]:
ratings_df = ratings_df.drop("timestamp", axis=1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<hr>

# User-Based Collaborative Filtering

<hr>

# Item-Based Collaborative Filtering